In [4]:
!pip install duckdb seaborn

In [1]:
import duckdb
#import scipy.stats as stats
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#from scipy import special
#import numpy as np
#import plotly.express as px

In [2]:
df_projects = pd.DataFrame()

def process(task_id: str):

    global df_projects

    print(f"== {task_id} ==")

    df_reliability = duckdb.query(f"""
        with dates as (
            select date--, message
            , row_number() OVER (ORDER BY date) as row_num
            from "commits/{task_id}-commits.csv"
            where change_type in ('__label__corrective')
        ) 
        select date_diff('days', d1.date, d2.date) TBF from dates d1
        left join dates d2 on d2.row_num = d1.row_num + 1

    """).to_df()

    TBF_data = df_reliability['TBF'].dropna()

    MTBF = df_reliability['TBF'].mean()
    lambda_failure = 1 / MTBF
    def reliability(t):
        return np.exp(-lambda_failure * t)
    
    r_linha = {}
    r_linha['project'] = f"{task_id}"

    t_values = [1, 5, 10, 50, 100]
    reliability_values = [reliability(t) for t in t_values]
    for t, R_t in zip(t_values, reliability_values):
        print(f"Confiabilidade R({t}) = {R_t:.4f}")
        r_linha[f"R{t}"] = f"{R_t:.4f}"

    print(r_linha)

    r_linha_df = pd.DataFrame([r_linha])
    df_projects = pd.concat([df_projects, r_linha_df], ignore_index=True)

    df_reliability.to_csv(f"reliability/{task_id}_reliability.csv")

    t_days = 1

    df_reliability_file = duckdb.query(f"""
        WITH dates AS (
            SELECT date, filepath, 
                ROW_NUMBER() OVER (PARTITION BY filepath ORDER BY date) AS row_num
            FROM "commits/{task_id}-files.csv" f
            LEFT JOIN "commits/{task_id}-commits.csv" c ON f.commit_id = c.commit_id
            WHERE change_type IN ('__label__corrective')
        ) 
        SELECT d1.filepath, 
            MAX(d1.row_num) AS n_commits, 
            AVG(DATE_DIFF('days', d1.date, d2.date)) AS MTBF,
            1.0 / NULLIF(AVG(DATE_DIFF('days', d1.date, d2.date)), 0) AS lambda_failure,  -- Evita divisão por zero
            EXP(- (1.0 / NULLIF(AVG(DATE_DIFF('days', d1.date, d2.date)), 0)) * {t_days}) AS R_t  -- Confiabilidade para t dias
        FROM dates d1
        LEFT JOIN dates d2 
            ON d2.filepath = d1.filepath 
            AND d2.row_num = d1.row_num + 1
        GROUP BY d1.filepath
        HAVING d1.filepath LIKE '%.java' AND MTBF > 0
        ORDER BY R_t, n_commits    
    """).to_df()

    with pd.option_context('display.max_rows', 20, 'display.max_columns', None,'display.max_colwidth', 100):
        display(df_reliability_file)    

    df_reliability_file.to_csv(f"reliability/{task_id}_reliability_file.csv")

#process('antlr4')
#df_projects.to_csv(f"reliability_projects.csv")


In [3]:
import os
log_dir = "logs/"
for filename in os.listdir(log_dir):
    if filename.endswith("_all_files.csv"):
        task_id = filename.replace("_all_files.csv", "")
        process(task_id)

df_projects.to_csv(f"reliability_projects.csv")

== logging-log4j2 ==
Confiabilidade R(1) = 0.7131
Confiabilidade R(5) = 0.1844
Confiabilidade R(10) = 0.0340
Confiabilidade R(50) = 0.0000
Confiabilidade R(100) = 0.0000
{'project': 'logging-log4j2', 'R1': '0.7131', 'R5': '0.1844', 'R10': '0.0340', 'R50': '0.0000', 'R100': '0.0000'}


,filepath,n_commits,MTBF,lambda_failure,R_t
0,log4j-core/src/test/java/org/apache/logging/log4j/core/appender/CsvJsonParameterLayoutFileAppend...,6,0.200000,5.000000,0.006738
1,log4j-mongodb/src/test/java/org/apache/logging/log4j/mongodb/MongoDbMapMessageTestJava8.java,3,0.500000,2.000000,0.135335
2,log4j-core/src/main/java/org/apache/logging/log4j/core/appender/db/jdbc/JdbcAppender.java,3,0.500000,2.000000,0.135335
3,log4j-core/src/test/java/org/apache/logging/log4j/core/appender/rolling/RollingAppenderDeleteAcc...,4,0.666667,1.500000,0.223130
4,log4j-core/src/test/java/org/apache/logging/log4j/core/appender/rolling/RollingAppenderDeleteAcc...,4,0.666667,1.500000,0.223130
...,...,...,...,...,...
782,log4j-api/src/test/java/org/apache/logging/log4j/MarkerTest.java,2,2710.000000,0.000369,0.999631
783,log4j-core/src/test/java/org/apache/logging/log4j/core/net/SocketMessageLossTest.java,2,2714.000000,0.000368,0.999632
784,log4j-core/src/main/java/org/apache/logging/log4j/core/appender/TlsSyslogFrame.java,2,3409.000000,0.000293,0.999707
785,log4j-api/src/main/java/org/apache/logging/log4j/message/StructuredDataMessage.java,2,3630.000000,0.000275,0.999725


== validation ==
Confiabilidade R(1) = 0.9916
Confiabilidade R(5) = 0.9585
Confiabilidade R(10) = 0.9187
Confiabilidade R(50) = 0.6546
Confiabilidade R(100) = 0.4285
{'project': 'validation', 'R1': '0.9916', 'R5': '0.9585', 'R10': '0.9187', 'R50': '0.6546', 'R100': '0.4285'}


,filepath,n_commits,MTBF,lambda_failure,R_t
0,src/main/java/javax/validation/metadata/BeanDescriptor.java,2,1.000000,1.000000,0.367879
1,src/main/java/javax/validation/ConstraintViolationException.java,2,4.000000,0.250000,0.778801
2,src/main/java/javax/validation/groups/Default.java,2,4.000000,0.250000,0.778801
3,src/main/java/javax/validation/metadata/ConstraintDescriptor.java,2,4.000000,0.250000,0.778801
4,src/main/java/javax/validation/bootstrap/ProviderSpecificBootstrap.java,2,4.000000,0.250000,0.778801
5,src/main/java/javax/validation/executable/ExecutableValidator.java,3,12.000000,0.083333,0.920044
6,src/main/java/javax/validation/ConstraintViolation.java,4,95.333333,0.010490,0.989565
7,src/main/java/javax/validation/Constraint.java,4,96.666667,0.010345,0.989708
8,src/main/java/javax/validation/ReportAsSingleViolation.java,2,143.000000,0.006993,0.993031
9,src/main/java/javax/validation/ConstraintValidatorContext.java,3,154.500000,0.006472,0.993548


== java-classmate ==
Confiabilidade R(1) = 0.9922
Confiabilidade R(5) = 0.9614
Confiabilidade R(10) = 0.9242
Confiabilidade R(50) = 0.6744
Confiabilidade R(100) = 0.4548
{'project': 'java-classmate', 'R1': '0.9922', 'R5': '0.9614', 'R10': '0.9242', 'R50': '0.6744', 'R100': '0.4548'}


...

,filepath,n_commits,MTBF,lambda_failure,R_t
0,examples/src/test/java/org/reactivestreams/example/unicast/AsyncSubscriberTest.java,4,4.666667,0.214286,0.807118
1,tck/src/main/java/org/reactivestreams/tck/support/HelperPublisher.java,2,5.000000,0.200000,0.818731
2,tck/src/main/java/org/reactivestreams/tck/support/HelperPublisherException.java,2,5.000000,0.200000,0.818731
3,tck/src/main/java/org/reactivestreams/tck/support/InfiniteHelperPublisher.java,2,5.000000,0.200000,0.818731
4,tck/src/test/java/org/reactivestreams/tck/support/TCKVerificationSupport.java,2,7.000000,0.142857,0.866878
5,tck/src/main/java/org/reactivestreams/tck/WithHelperPublisher.java,3,7.000000,0.142857,0.866878
6,tck/src/test/java/org/reactivestreams/tck/IdentityProcessorVerificationTest.java,4,28.666667,0.034884,0.965718
7,tck/src/main/java/org/reactivestreams/tck/support/SubscriberWhiteboxVerificationRules.java,2,75.000000,0.013333,0.986755
8,tck/src/main/java/org/reactivestreams/tck/IdentityProcessorVerification.java,13,98.916667,0.010110,0.989941
9,tck/src/main/java/org/reactivestreams/tck/PublisherVerification.java,14,151.384615,0.006606,0.993416


== spring-petclinic ==
Confiabilidade R(1) = 0.9825
Confiabilidade R(5) = 0.9155
Confiabilidade R(10) = 0.8381
Confiabilidade R(50) = 0.4136
Confiabilidade R(100) = 0.1711
{'project': 'spring-petclinic', 'R1': '0.9825', 'R5': '0.9155', 'R10': '0.8381', 'R50': '0.4136', 'R100': '0.1711'}


,filepath,n_commits,MTBF,lambda_failure,R_t
0,src/main/java/org/springframework/samples/petclinic/service/ClinicServiceImpl.java,2,1.000000,1.000000,0.367879
1,src/main/java/org/springframework/samples/petclinic/repository/jpa/JpaPetRepositoryImpl.java,2,1.000000,1.000000,0.367879
2,src/test/java/org/springframework/samples/petclinic/web/VisitsAtomViewTest.java,2,3.000000,0.333333,0.716531
3,src/main/java/org/springframework/samples/petclinic/Pet.java,2,7.000000,0.142857,0.866878
4,src/test/java/org/springframework/samples/petclinic/AbstractPetRepositoryTests.java,2,14.000000,0.071429,0.931063
5,src/test/java/org/springframework/samples/petclinic/AbstractOwnerRepositoryTests.java,2,14.000000,0.071429,0.931063
6,src/main/java/org/springframework/samples/petclinic/Vet.java,2,24.000000,0.041667,0.959189
7,src/main/java/org/springframework/samples/petclinic/repository/jpa/JpaOwnerRepositoryImpl.java,4,32.666667,0.030612,0.969852
8,src/main/java/org/springframework/samples/petclinic/repository/jpa/JpaVisitRepositoryImpl.java,3,39.000000,0.025641,0.974685
9,src/main/java/org/springframework/samples/petclinic/repository/jpa/JpaVetRepositoryImpl.java,2,98.000000,0.010204,0.989848


== spring-boot ==
Confiabilidade R(1) = 0.5980
Confiabilidade R(5) = 0.0765
Confiabilidade R(10) = 0.0059
Confiabilidade R(50) = 0.0000
Confiabilidade R(100) = 0.0000
{'project': 'spring-boot', 'R1': '0.5980', 'R5': '0.0765', 'R10': '0.0059', 'R50': '0.0000', 'R100': '0.0000'}


,filepath,n_commits,MTBF,lambda_failure,R_t
0,spring-boot-project/spring-boot-tools/spring-boot-maven-plugin/src/intTest/projects/build-image-...,3,0.5,2.000000,0.135335
1,spring-boot-project/spring-boot-tools/spring-boot-loader-tools/src/main/java/org/springframework...,3,0.5,2.000000,0.135335
2,spring-boot-actuator/src/main/java/org/springframework/boot/actuate/autoconfigure/SecurityAutoCo...,2,1.0,1.000000,0.367879
3,spring-boot-project/spring-boot/src/test/java/org/springframework/boot/logging/log4j2/Log4j2File...,2,1.0,1.000000,0.367879
4,spring-boot/src/main/java/org/springframework/boot/context/properties/bind/Binder.java,2,1.0,1.000000,0.367879
...,...,...,...,...,...
1038,spring-boot-project/spring-boot-actuator/src/main/java/org/springframework/boot/actuate/audit/Au...,2,2548.0,0.000392,0.999608
1039,spring-boot-project/spring-boot-test/src/test/java/org/springframework/boot/test/context/assertj...,2,2591.0,0.000386,0.999614
1040,spring-boot-project/spring-boot-test/src/main/java/org/springframework/boot/test/util/TestProper...,2,2621.0,0.000382,0.999619
1041,spring-boot-project/spring-boot-actuator/src/main/java/org/springframework/boot/actuate/logging/...,2,2621.0,0.000382,0.999619
